# Machine Learning

## Support Vector Machine

## Let's start with a simple dataset that is linearly separable

## 1. Scratch: SVM with Hard Margin


### Compute w and b parameters

## 2. Scratch: SVM with Soft Margin

### Comparing to Sklearn results

## 3. Scratch: Implementing Kernels